# Dataset Exploration and Validation
## PaySim Synthetic Financial Fraud Detection Dataset

**Objective**: Validate dataset structure, understand distributions, and verify alignment with PRD constraints.

**Key Constraints**:
- ❌ Cannot use balance columns (oldbalanceOrg, newbalanceOrig, oldbalanceDest, newbalanceDest)
- ✅ Available features: step, type, amount, nameOrig, nameDest
- ✅ Target: isFraud
- ✅ Rule indicator: isFlaggedFraud

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# Plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Libraries imported")

## 1. Load Dataset

In [ ]:
# Load dataset
dataset_path = Path('../dataset/Synthetic_Financial_datasets_log.csv')
print(f"Loading dataset from: {dataset_path}")
print(f"File exists: {dataset_path.exists()}")
print(f"File size: {dataset_path.stat().st_size / (1024**2):.2f} MB\n")

# Read CSV
df = pd.read_csv(dataset_path)

print(f"✅ Dataset loaded successfully")
print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")

## 2. Basic Structure Validation

In [ ]:
# Display first few rows
print("First 5 rows:")
df.head()

In [ ]:
# Column info
print("Dataset Info:")
df.info()

In [ ]:
# Expected columns from PRD
expected_columns = [
    'step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
    'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud', 'isFlaggedFraud'
]

available_features = ['step', 'type', 'amount', 'nameOrig', 'nameDest']
restricted_features = ['oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']
target_labels = ['isFraud', 'isFlaggedFraud']

print("Column Validation:")
print(f"Expected columns: {len(expected_columns)}")
print(f"Actual columns: {len(df.columns)}")
print(f"\nMissing columns: {set(expected_columns) - set(df.columns)}")
print(f"Extra columns: {set(df.columns) - set(expected_columns)}")
print(f"\n✅ Available features (can use): {available_features}")
print(f"❌ Restricted features (cannot use): {restricted_features}")
print(f"🎯 Target labels: {target_labels}")

## 3. Data Quality Assessment

In [ ]:
# Missing values
print("Missing Values:")
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Percentage': missing_pct
})
missing_df[missing_df['Missing Count'] > 0]

In [ ]:
# Duplicates
duplicates = df.duplicated().sum()
print(f"Duplicate rows: {duplicates:,} ({duplicates/len(df)*100:.4f}%)")

# Check for duplicate transactions (may be valid)
dup_by_key = df.duplicated(subset=['step', 'type', 'amount', 'nameOrig', 'nameDest']).sum()
print(f"Duplicate by transaction key: {dup_by_key:,}")

In [ ]:
# Statistical summary
print("Statistical Summary (Numerical Features):")
df.describe()

## 4. Feature Analysis: Available Features Only

In [ ]:
# Step (time dimension)
print("=" * 60)
print("STEP (Time Dimension)")
print("=" * 60)
print(f"Min step: {df['step'].min()}")
print(f"Max step: {df['step'].max()}")
print(f"Unique steps: {df['step'].nunique()}")
print(f"Range: {df['step'].min()} to {df['step'].max()} (Expected: 1-744)")
print(f"\nTransactions per step (hour):")
print(df.groupby('step').size().describe())

In [ ]:
# Type (transaction type)
print("=" * 60)
print("TYPE (Transaction Type)")
print("=" * 60)
print(f"Unique types: {df['type'].nunique()}")
print(f"\nTransaction type distribution:")
type_dist = df['type'].value_counts().sort_index()
type_pct = (type_dist / len(df) * 100).round(2)
pd.DataFrame({
    'Count': type_dist,
    'Percentage': type_pct
})

In [ ]:
# Amount
print("=" * 60)
print("AMOUNT")
print("=" * 60)
print(f"Min amount: {df['amount'].min():,.2f}")
print(f"Max amount: {df['amount'].max():,.2f}")
print(f"Mean amount: {df['amount'].mean():,.2f}")
print(f"Median amount: {df['amount'].median():,.2f}")
print(f"Std amount: {df['amount'].std():,.2f}")
print(f"\nZero amounts: {(df['amount'] == 0).sum():,}")
print(f"Negative amounts: {(df['amount'] < 0).sum():,}")

In [ ]:
# nameOrig (sender)
print("=" * 60)
print("nameOrig (Sender/Origin)")
print("=" * 60)
print(f"Unique originators: {df['nameOrig'].nunique():,}")
print(f"Total transactions: {len(df):,}")
print(f"Avg transactions per originator: {len(df) / df['nameOrig'].nunique():.2f}")
print(f"\nTop 10 most active originators:")
df['nameOrig'].value_counts().head(10)

In [ ]:
# nameDest (recipient)
print("=" * 60)
print("nameDest (Recipient/Destination)")
print("=" * 60)
print(f"Unique recipients: {df['nameDest'].nunique():,}")
print(f"Total transactions: {len(df):,}")
print(f"Avg transactions per recipient: {len(df) / df['nameDest'].nunique():.2f}")
print(f"\nTop 10 most active recipients:")
df['nameDest'].value_counts().head(10)

In [ ]:
# Entity ID patterns (C = Customer, M = Merchant)
print("=" * 60)
print("Entity ID Patterns")
print("=" * 60)

def analyze_entity_prefix(series, name):
    prefixes = series.str[0].value_counts()
    print(f"\n{name} prefixes:")
    for prefix, count in prefixes.items():
        print(f"  {prefix}: {count:,} ({count/len(series)*100:.2f}%)")

analyze_entity_prefix(df['nameOrig'], 'nameOrig')
analyze_entity_prefix(df['nameDest'], 'nameDest')

## 5. Target Variable Analysis

In [ ]:
# isFraud distribution
print("=" * 60)
print("isFraud (Target Variable)")
print("=" * 60)
fraud_dist = df['isFraud'].value_counts().sort_index()
fraud_pct = (fraud_dist / len(df) * 100).round(4)

fraud_df = pd.DataFrame({
    'Count': fraud_dist,
    'Percentage': fraud_pct
})
fraud_df.index = ['Not Fraud', 'Fraud']
print(fraud_df)

print(f"\n⚠️ Class Imbalance Ratio: 1:{(fraud_dist[0] / fraud_dist[1]):.0f}")

In [ ]:
# isFlaggedFraud (high-value transfer rule)
print("=" * 60)
print("isFlaggedFraud (Rule: TRANSFER > 200,000)")
print("=" * 60)
flagged_dist = df['isFlaggedFraud'].value_counts().sort_index()
flagged_pct = (flagged_dist / len(df) * 100).round(4)

flagged_df = pd.DataFrame({
    'Count': flagged_dist,
    'Percentage': flagged_pct
})
flagged_df.index = ['Not Flagged', 'Flagged']
print(flagged_df)

In [ ]:
# Relationship between isFraud and isFlaggedFraud
print("=" * 60)
print("Confusion Matrix: isFlaggedFraud vs isFraud")
print("=" * 60)
confusion = pd.crosstab(
    df['isFlaggedFraud'], 
    df['isFraud'], 
    rownames=['isFlaggedFraud'], 
    colnames=['isFraud'],
    margins=True
)
print(confusion)

# Check if flagged transactions are actually fraud
flagged_txns = df[df['isFlaggedFraud'] == 1]
if len(flagged_txns) > 0:
    flagged_fraud_rate = flagged_txns['isFraud'].mean() * 100
    print(f"\nPrecision of isFlaggedFraud rule: {flagged_fraud_rate:.2f}%")

## 6. Fraud Analysis by Transaction Type

In [ ]:
# Fraud by type
print("=" * 60)
print("Fraud Distribution by Transaction Type")
print("=" * 60)

fraud_by_type = df.groupby('type').agg({
    'isFraud': ['sum', 'mean', 'count']
}).round(4)
fraud_by_type.columns = ['Fraud Count', 'Fraud Rate', 'Total Transactions']
fraud_by_type['Fraud Rate'] = (fraud_by_type['Fraud Rate'] * 100).round(2)
fraud_by_type = fraud_by_type.sort_values('Fraud Rate', ascending=False)
print(fraud_by_type)

print("\n⚠️ Key Insight: Fraud occurs only in specific transaction types")

In [ ]:
# Amount distribution for fraud vs non-fraud
print("=" * 60)
print("Amount Statistics: Fraud vs Non-Fraud")
print("=" * 60)

amount_stats = df.groupby('isFraud')['amount'].describe()
amount_stats.index = ['Not Fraud', 'Fraud']
print(amount_stats)

## 7. Visualizations

In [ ]:
# Transaction type distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Count
df['type'].value_counts().plot(kind='bar', ax=axes[0], color='steelblue')
axes[0].set_title('Transaction Type Distribution (Count)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Transaction Type')
axes[0].set_ylabel('Count')
axes[0].tick_params(axis='x', rotation=45)

# Percentage
(df['type'].value_counts() / len(df) * 100).plot(kind='bar', ax=axes[1], color='coral')
axes[1].set_title('Transaction Type Distribution (%)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Transaction Type')
axes[1].set_ylabel('Percentage')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Fraud distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Class imbalance
fraud_counts = df['isFraud'].value_counts()
axes[0].pie(fraud_counts, labels=['Not Fraud', 'Fraud'], autopct='%1.2f%%', colors=['lightgreen', 'red'])
axes[0].set_title('Class Distribution: isFraud', fontsize=14, fontweight='bold')

# Fraud by transaction type
fraud_by_type_plot = df.groupby('type')['isFraud'].mean() * 100
fraud_by_type_plot.plot(kind='bar', ax=axes[1], color='indianred')
axes[1].set_title('Fraud Rate by Transaction Type', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Transaction Type')
axes[1].set_ylabel('Fraud Rate (%)')
axes[1].tick_params(axis='x', rotation=45)
axes[1].axhline(y=df['isFraud'].mean() * 100, color='black', linestyle='--', label='Overall Rate')
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Amount distribution (log scale)
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Non-fraud
non_fraud = df[df['isFraud'] == 0]['amount']
axes[0].hist(np.log10(non_fraud[non_fraud > 0]), bins=50, color='lightblue', edgecolor='black', alpha=0.7)
axes[0].set_title('Amount Distribution: Not Fraud (log10 scale)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('log10(Amount)')
axes[0].set_ylabel('Frequency')

# Fraud
fraud = df[df['isFraud'] == 1]['amount']
axes[1].hist(np.log10(fraud[fraud > 0]), bins=50, color='salmon', edgecolor='black', alpha=0.7)
axes[1].set_title('Amount Distribution: Fraud (log10 scale)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('log10(Amount)')
axes[1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
# Transactions over time (steps)
fig, axes = plt.subplots(2, 1, figsize=(15, 8))

# All transactions
step_counts = df.groupby('step').size()
axes[0].plot(step_counts.index, step_counts.values, color='steelblue', linewidth=1)
axes[0].set_title('Transaction Volume Over Time', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Step (Hour)')
axes[0].set_ylabel('Transaction Count')
axes[0].grid(True, alpha=0.3)

# Fraud transactions
fraud_by_step = df[df['isFraud'] == 1].groupby('step').size()
axes[1].plot(fraud_by_step.index, fraud_by_step.values, color='red', linewidth=1)
axes[1].set_title('Fraud Transaction Volume Over Time', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Step (Hour)')
axes[1].set_ylabel('Fraud Count')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Data Validation Summary

In [ ]:
print("=" * 80)
print("DATA VALIDATION SUMMARY")
print("=" * 80)

validations = [
    ("✅", "All expected columns present", set(expected_columns) == set(df.columns)),
    ("✅", "No missing values", df.isnull().sum().sum() == 0),
    ("✅", "Step range valid (1-744)", df['step'].min() == 1 and df['step'].max() == 744),
    ("✅", "Transaction types valid", df['type'].nunique() == 5),
    ("✅", "No negative amounts", (df['amount'] < 0).sum() == 0),
    ("✅", "High cardinality entities", df['nameOrig'].nunique() > 1000 and df['nameDest'].nunique() > 1000),
    ("⚠️", "Severe class imbalance", (df['isFraud'].sum() / len(df)) < 0.01),
    ("✅", "Fraud only in specific types", df.groupby('type')['isFraud'].sum().gt(0).sum() <= 3),
]

for symbol, description, condition in validations:
    print(f"{symbol} {description}")

print("\n" + "=" * 80)
print("KEY FINDINGS")
print("=" * 80)
print(f"📊 Dataset Size: {len(df):,} transactions")
print(f"📅 Time Range: {df['step'].nunique()} steps (hours)")
print(f"👥 Unique Originators: {df['nameOrig'].nunique():,}")
print(f"🏪 Unique Recipients: {df['nameDest'].nunique():,}")
print(f"💰 Amount Range: {df['amount'].min():,.2f} to {df['amount'].max():,.2f}")
print(f"🚨 Fraud Rate: {df['isFraud'].mean() * 100:.4f}%")
print(f"⚖️ Class Imbalance: 1:{(df['isFraud'].value_counts()[0] / df['isFraud'].value_counts()[1]):.0f}")

fraud_types = df[df['isFraud'] == 1]['type'].unique()
print(f"🎯 Fraud occurs in: {', '.join(fraud_types)}")

print("\n" + "=" * 80)
print("NEXT STEPS")
print("=" * 80)
print("1. ✅ Dataset structure validated")
print("2. 🔄 Design feature engineering strategy (behavioral features only)")
print("3. 🔄 Implement time-based train/val/test split")
print("4. 🔄 Handle class imbalance (sampling, weights, metrics)")
print("5. 🔄 Baseline model development")